In [1]:
import torch 
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
online_mode = False

In [2]:
import mediapy

# read a video into a tensor
video = torch.tensor(mediapy.read_video("/home/manuelf/mast3r/images_in/apple.mp4")).permute(0,3,1,2).unsqueeze(0).to(device).float()

In [ ]:
#cotracker inference
grid_size = 10

if online_mode:
    cotracker_model = torch.hub.load("facebookresearch/co-tracker", "cotracker3_online").to(device)

    cotracker_model(video_chunk=video, is_first_step=True, grid_size=grid_size)  

    for ind in range(0, video.shape[1] - cotracker_model.step, cotracker_model.step):
        pred_tracks, pred_visibility = cotracker_model(
            video_chunk=video[:, ind : ind + cotracker_model.step * 2]
        )  # B T N 2,  B T N 1

else:
    cotracker_model = torch.hub.load("facebookresearch/co-tracker", "cotracker3_offline").to(device)

    pred_tracks, pred_visibility = cotracker_model(video, grid_size=grid_size) # B T N 2,  B T N 1

Using cache found in /home/manuelf/.cache/torch/hub/facebookresearch_co-tracker_main


In [ ]:
# save the data as mat-files to be read in matlab
from scipy.io import savemat

savemat("cotracker_tracks.mat", {
    "tracks": pred_tracks.cpu().numpy(),
    "visibility": pred_visibility.cpu().numpy(),
})


In [6]:
from cotracker.utils.visualizer import Visualizer

vis = Visualizer(save_dir="./saved_videos", pad_value=120, linewidth=3)
vis.visualize(video, pred_tracks, pred_visibility)

Video saved to ./saved_videos/video.mp4


tensor([[[[[255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           ...,
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255]],

          [[255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           ...,
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255]],

          [[255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           ...,
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255],
           [255, 255, 255,  ..., 255, 255, 255]]],


         [[[255, 255, 255,  ..., 255, 255, 255],
           [2